In [9]:
import pennylane as qml
from pennylane import numpy as np

In [10]:
#GAME CODE
def game(nx_qubits,ny_qubits,level,ini,state=[],game_card={}):
    if ini==-1:
        dev1 = qml.device("default.qubit",wires=range(nx_qubits*ny_qubits))
        @qml.qnode(dev1)
        def ini_circuit():
            n_total = nx_qubits*ny_qubits
            coords = np.random.choice(n_total,level, replace=False)
            for coord in coords:
                qml.PauliX(wires=coord)
            return qml.state()
        return ini_circuit()
    elif ini==-2: 
        dev2 = qml.device("default.qubit",wires=range(nx_qubits*ny_qubits))
        @qml.qnode(dev2)
        def intermediate_circuit():
            qml.AmplitudeEmbedding(state, wires=range(nx_qubits*ny_qubits))
            for swap in game_card['SWAP']:
                qml.SWAP(wires=list(map(int,swap.split(','))))
            for hospital in game_card['hospital']:
                if int(hospital) == (nx_qubits*ny_qubits-1)/2:
                    print("Welcome to the most dangerous, but efficacious hospital.")
                    qml.PauliX(wires=hospital)
                else:
                    print("Here, this is a medicine. But I'm not sure it may help you heal completely...")
                    qml.Hadamard(wires=hospital)
            return qml.state()
        return intermediate_circuit()
    elif ini == -3:
        dev3 = qml.device("default.qubit",shots=1,wires=range(nx_qubits*ny_qubits))
        @qml.qnode(dev3)
        def end_circuit():
            qml.AmplitudeEmbedding(state, wires=range(nx_qubits*ny_qubits))
            return qml.sample()
        final_pcr = end_circuit()
        if 1 not in final_pcr:
            return 'All citizens are cured!!! YOU WIN'
        else:
            return 'Virus is not disappeared. YOU LOSE'
    elif ini == -4:
        dev6 = qml.device("default.qubit",shots=1,wires=range(nx_qubits*ny_qubits))
        @qml.qnode(dev6)
        def total_survey_circuit():
            qml.AmplitudeEmbedding(state, wires=range(nx_qubits*ny_qubits))
            return qml.sample()
        dev7 = qml.device("default.qubit",wires=range(nx_qubits*ny_qubits))
        @qml.qnode(dev7)
        def after_survey(total_survey):
            print("The entire PCR test result is:",total_survey)
            qml.BasisState(total_survey, wires=range(nx_qubits*ny_qubits))
            return qml.state()
        return after_survey(total_survey_circuit())
    else:
        dev4 = qml.device("default.qubit",shots=1,wires=range(nx_qubits*ny_qubits))
        @qml.qnode(dev4)
        def measure_circuit(issample):
            qml.AmplitudeEmbedding(state, wires=range(nx_qubits*ny_qubits))
            if issample:
                return qml.sample(wires=ini)
            else:
                return qml.state()
        measure_out = measure_circuit(True)
        if measure_out == 1:
            print("The PCR test of person ",ini," is ","Positive.")
        else:
            print("The PCR test of person ",ini," is ","Negative.")
        states = measure_circuit(False)
        aft_meas = []
        j =nx_qubits*ny_qubits - 1 - ini
        for i, st in enumerate(states):
            indx = int((i % (2**(j+1)))/2**j)
            if measure_out == indx:
                aft_meas.append(st)
            else :
                aft_meas.append(0)
        dev5 = qml.device("default.qubit",wires=range(nx_qubits*ny_qubits))
        @qml.qnode(dev5)
        def re_input_circuit():
            qml.AmplitudeEmbedding(aft_meas, wires=range(nx_qubits*ny_qubits),normalize=True)
            return qml.state()
        return re_input_circuit()
            
        

In [12]:
#INPUT AND OUTPUT
nx_qubits = int(input("number of columns:"))
ny_qubits = int(input("number of rows:"))
level = int(input("level:"))
ini_bool = -1
ini_result = game(nx_qubits,ny_qubits,level,ini_bool)
print("Quandemic!! It's time to rescue human")
progress = True
can_total_survey = True
while progress:
    swaps = input("pairs of infection:").split()
    swap_test = True
    while swap_test:
        for swap in swaps:
            i = list(map(int,swap.split(',')))[0]
            j = list(map(int,swap.split(',')))[1]
            if np.abs(i-j) == nx_qubits or np.abs(i-j) == 1 and not (i+j+1)%(2*nx_qubits) == 0:
                pass
            else:
                print("You may insert invalid pairs of infection. Retry.")
                swaps = input("pairs of infection:").split()
                swap_test = True
                break
            swap_test = False
    hospital = list(map(int,input("select the hospitals:").split()))
    hospital_test = True
    while hospital_test:
        for hos in hospital:
            if hos == (nx_qubits*ny_qubits-1)/2 or hos in range(nx_qubits) or hos in range(nx_qubits*ny_qubits-nx_qubits,nx_qubits*ny_qubits) or hos in range(0,nx_qubits*ny_qubits,nx_qubits) or hos in range(nx_qubits-1,nx_qubits*ny_qubits,nx_qubits):
                pass
            else:
                print("You may insert invalid hospitals. Retry.")
                hospital = map(int,input("select the hospitals:").split())
                hospital_test = True
                break
            hospital_test = False
    game_card = {}
    game_card['SWAP'] = swaps
    game_card['hospital'] = hospital
    ini_bool = -2
    ini_result = game(nx_qubits,ny_qubits,level,ini_bool,ini_result,game_card)
    ini_bool = int(input("Who wants to get pcr?:")) # ini_bool for index of person who gets PCR test
    ini_result = game(nx_qubits,ny_qubits,level,ini_bool,ini_result)
    if can_total_survey:
        total_survey = int(input("You have a chance to test all people. Do you want to use the chance?:"))
        if total_survey==1:
            ini_result = game(nx_qubits,ny_qubits,level,-4,ini_result)
            can_total_survey = False
    apocalyptic = np.abs(int(input("Would you like to check out the result?:"))-1)
    if apocalyptic == 0:
        print(game(nx_qubits,ny_qubits,level,-3,ini_result))
    progress = apocalyptic

number of columns:3
number of rows:3
level:1
Quandemic!! It's time to rescue human
pairs of infection:2 1


IndexError: list index out of range